# Simple gaffer python demo

## Setup

In [ ]:
from gafferpy import gaffer as g
from gafferpy import gaffer_connector
import json
from pprint import pprint

In [ ]:
host = "http://localhost:8080/rest/latest"

In [ ]:
gc = gaffer_connector.GafferConnector(host)

## Get Schema

In [ ]:
schema = gc.execute_get(
    g.GetSchema()
)

print(json.dumps(json.loads(schema), indent=4))

## Get Elements

In [ ]:
input = gc.execute_operation(
    g.GetElements(
        input=['M32:1'],
        view=g.View(
            edges=[
                g.ElementDefinition(
                    group='RoadUse',
                    group_by=[]
                )
            ]
        )
    )
)
pprint(input)

In [ ]:
input = gc.execute_operation(
    g.GetElements(
        input=['M32:1'],
        view=g.View(
            edges=[
                g.ElementDefinition(
                    group='RoadUse',
                    group_by=[],
                    pre_aggregation_filter_functions=[
                        g.PredicateContext(
                            selection=['count'],
                            predicate=g.IsMoreThan(
                                value=g.long(1)
                            )
                        )
                    ]
                )
            ]
        ),
        directed_type=g.DirectedType.EITHER
    )
)
print('Related input')
pprint(input)
print()

In [ ]:
input = gc.execute_operation(
    g.GetElements(
        input=['M32:1'],
        view=g.View(
            edges=[
                g.ElementDefinition(
                    group='RoadUse',
                    group_by=[],
                    transient_properties=[
                        g.Property('description', 'java.lang.String')
                    ],
                    transform_functions=[
                        g.FunctionContext(
                            selection=['SOURCE', 'DESTINATION', 'count'],
                            function=g.Function(
                                class_name='uk.gov.gchq.gaffer.traffic.transform.DescriptionTransform'
                            ),
                            projection=['description']
                        )
                    ],
                    properties=['description']
                )
            ]
        ),
        directed_type=g.DirectedType.EITHER
    )
)
print('Related input')
pprint(input)
print()

## Operation Chain
Get junctions in South West that had the most cars in the year 2000.

In [ ]:
operation_chain=g.OperationChain( 
    operations=[ 
      g.GetAdjacentIds(
        input=[g.EntitySeed("South West")],
        view=g.View( 
          edges=[ 
            g.ElementDefinition( 
              group="RegionContainsLocation" 
            ) 
          ] 
        ) 
      ), 
      g.GetAdjacentIds( 
        view=g.View( 
          edges=[ 
            g.ElementDefinition( 
              group="LocationContainsRoad" 
            ) 
          ] 
        ) 
      ), 
      g.GetAdjacentIds( 
        view=g.View( 
          edges=[ 
            g.ElementDefinition( 
              group="RoadHasJunction" 
            ) 
          ] 
        ) 
      ), 
      g.GetElements( 
        view=g.View( 
          entities=[ 
            g.ElementDefinition( 
              transform_functions=[ 
                g.FunctionContext( 
                  projection=[ 
                    "CAR" 
                  ], 
                  function=g.FreqMapExtractor( 
                    key="CAR" 
                  ), 
                  selection=[ 
                    "countByVehicleType" 
                  ] 
                ) 
              ], 
              group="JunctionUse", 
              transient_properties={'CAR': 'Long'}, 
              properties=[ 
                "CAR" 
              ], 
              pre_aggregation_filter_functions=[ 
                g.PredicateContext( 
                  predicate=g.InDateRangeDual( 
                    start="2000/01/01", 
                    end="2001/01/01" 
                  ), 
                  selection=[ 
                    "startDate", 
                    "endDate" 
                  ] 
                ) 
              ] 
            ) 
          ], 
          global_elements=[ 
            g.GlobalElementDefinition( 
              group_by=[ 
              ] 
            ) 
          ] 
        ), 
        include_incoming_out_going="OUTGOING" 
      ), 
      g.Sort( 
        result_limit=5, 
        comparators=[ 
          g.ElementPropertyComparator( 
            reversed=True, 
            groups=[ 
              "JunctionUse" 
            ], 
            property="CAR" 
          ) 
        ], 
        deduplicate=True 
      ), 
      g.If( 
        then=g.ToCsv( 
          include_header=True, 
          element_generator=g.CsvGenerator( 
            quoted=False, 
            constants={}, 
            comma_replacement=" ", 
            fields={'VERTEX': 'Junction', 'CAR': 'CAR'} 
          ) 
        ), 
        condition=True 
      ) 
    ]
)

gc.execute_operation(operation_chain)

## Create Named Operation
Create a named operation based on the previous operation chain

In [ ]:
named_op = g.AddNamedOperation( 
  operation_name="frequent-vehicles-in-region", 
  description="Finds the junctions in a region with the most of an individual vehicle (e.g BUS, CAR) in the year 2000. The input is the region.", 
  write_access_roles=[ 
  ], 
  overwrite_flag=True, 
  operation_chain=g.OperationChain( 
    operations=[ 
      g.GetAdjacentIds( 
        view=g.View( 
          edges=[ 
            g.ElementDefinition( 
              group="RegionContainsLocation" 
            ) 
          ] 
        ) 
      ), 
      g.GetAdjacentIds( 
        view=g.View( 
          edges=[ 
            g.ElementDefinition( 
              group="LocationContainsRoad" 
            ) 
          ] 
        ) 
      ), 
      g.GetAdjacentIds( 
        view=g.View( 
          edges=[ 
            g.ElementDefinition( 
              group="RoadHasJunction" 
            ) 
          ] 
        ) 
      ), 
      g.GetElements( 
        view=g.View( 
          entities=[ 
            g.ElementDefinition( 
              transform_functions=[ 
                g.FunctionContext( 
                  projection=[ 
                    "${vehicle}" 
                  ], 
                  function=g.FreqMapExtractor( 
                    key="${vehicle}" 
                  ), 
                  selection=[ 
                    "countByVehicleType" 
                  ] 
                ) 
              ], 
              group="JunctionUse", 
              transient_properties={'${vehicle}': 'Long'}, 
              properties=[ 
                "${vehicle}" 
              ], 
              pre_aggregation_filter_functions=[ 
                g.PredicateContext( 
                  predicate=g.InDateRangeDual( 
                    start="2000/01/01", 
                    end="2001/01/01" 
                  ), 
                  selection=[ 
                    "startDate", 
                    "endDate" 
                  ] 
                ) 
              ] 
            ) 
          ], 
          global_elements=[ 
            g.GlobalElementDefinition( 
              group_by=[ 
              ] 
            ) 
          ] 
        ), 
        include_incoming_out_going="OUTGOING" 
      ), 
      g.Sort( 
        result_limit="${result-limit}", 
        comparators=[ 
          g.ElementPropertyComparator( 
            reversed=True, 
            groups=[ 
              "JunctionUse" 
            ], 
            property="${vehicle}" 
          ) 
        ], 
        deduplicate=True 
      ), 
      g.If( 
        then=g.ToCsv( 
          include_header=True, 
          element_generator=g.CsvGenerator( 
            quoted=False, 
            constants={}, 
            comma_replacement=" ", 
            fields={'VERTEX': 'Junction', '${vehicle}': '${vehicle}'} 
          ) 
        ), 
        condition="${to-csv}" 
      ) 
    ] 
  ), 
  parameters=[ 
    g.NamedOperationParameter( 
      name="vehicle", 
      required=False, 
      description="The type of vehicle: HGVR3, BUS, HGVR4, AMV, HGVR2, HGVA3, PC, HGVA3, PC, HGCA5, HGVA6, CAR, HGV, WM2, LGV", 
      value_class="java.lang.String", 
      default_value="BUS" 
    ), 
    g.NamedOperationParameter( 
      name="result-limit", 
      required=False, 
      description="The maximum number of junctions to return", 
      value_class="java.lang.Integer", 
      default_value=2 
    ), 
    g.NamedOperationParameter( 
      name="to-csv", 
      required=False, 
      description="Enable this parameter to convert the results to a simple CSV in the format: Junction, Count", 
      value_class="java.lang.Boolean", 
      default_value=False 
    ) 
  ], 
  read_access_roles=[ 
  ] 
)

gc.execute_operation(named_op)

## Execute Named Operation

In [ ]:
gc.execute_operation(
    g.NamedOperation(
        operation_name="frequent-vehicles-in-region",
        input=[g.EntitySeed("South West")],
        parameters={
            'vehicle': 'CAR',
            'result-limit': 5,
            'to-csv': True
        }
    )
)